In [1]:
import csv
list=[]
f= open('title.csv','r',encoding="utf-8")
csv_reader = csv.reader(f)
for row in csv_reader:
    list.append(row[0])    
    

In [2]:
from gensim import corpora
dictionary = corpora.Dictionary([list])#方法遍历所有的文本，为每个不重复的单词分配一个单独的整数ID
print(dictionary)#[' ',' ',' '...]12220

#Dictionary 函数的补充
#dictionary.add_documents([['',''],['']])#更新词典，加入新的不重复词

#dictionary.doc2bow(['','',''])#输出：[(词ID，词频)..]，对该文本进行词袋格式的转换，查找词典在文档出现的词和次数进行输出

#dictionary.doc2idx(['','',''])#输出：[对应词在词典中的索引号，，]，不在词典中的词的索引号可通过属性unknow_word_index设置

#https://radimrehurek.com/gensim/corpora/dictionary.html更多有关函数可参考官网

#将词汇表写入文本vocab.txt中
f = open('vocab.txt','w')
#l为索引号
for l in dictionary:
    #print(dictionary[l])
    f.write(dictionary[l]+'\n')
f.close()


Dictionary(2351 unique tokens: ["'Brazilian style science' - an analysis of the difference between Brazilian and international Computer Science departments and graduate programs using social networks analysis and bibliometrics.", "'It could turn ugly': Selective disclosure of attitudes in political discussion networks.", "'It must be me' or 'It could be them?': The impact of the social network position of bullies and victims on victims\\' adjustment.", "'Think before you upload': an in-depth analysis of unavailable videos on YouTube.", "'Trust Me': Differences in expressed and perceived trust relations in an organization."]...)


In [3]:
corpus = [dictionary.doc2bow([doc]) for doc in list]
#第13个标题向量的结果
print(corpus[12])#[(5, 1), (64, 1), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1)] len(corpus[12])=7

#将结果按格式要求存入文本文件title.txt
#ti代表一条标题向量，len(ti)计算该向量的项数
f = open("title.txt","w")
for ti in corpus:
   #print(len(ti))
   f.write(str(len(ti))+' ')
   for i in range(len(ti)):
      # print(str(ti[i][0])+":"+str(ti[i][1]))
       f.write(str(ti[i][0])+":"+str(ti[i][1])+" ")
   f.write("\n")
f.close()


[(42, 1)]


In [4]:
import gensim

# 在 DT 矩阵上运行和训练 LDA 模型
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=5)#指定主题数为5

"""
#将模型保存到磁盘或重新加载预先训练的模型
temp_file='/users/sds'
model.save(temp_file)
model=Ldamodel.load(temp_file)
"""

#用模型训练新文档，分配主题
other_text=[['','','']]
other_corpus=dictionary.doc2bow([other_text])
vector=model[other_corpus]

#对新语料库进行增量培训来更新模型
model.update(other_corpus)

#获取单个主题及主题下的单词
print_topic(topicno,topn=k)#k为显示的单词数
print_topics(num_topics=n,num_words=k)#k同上，n为显示的主题数

#获取给定文档的主题分布
get_document_topics(bow,minimum_probability)#bow为词袋格式

corpus_lda = model[corpus]  # 每行标题对应5个主题的权重
print(len(corpus_lda))#30796行标题
print(corpus_lda[0])#[(0, 0.48325282), (1, 0.04041054), (2, 0.040259134), (3, 0.39596453), (4, 0.040113024)]

#获取每个单词的主题分布，每个词对应5个主题的权重,取权重最高的为其主题，存入字典d1
d1=dict()
#print(d1)
for l in dictionary:
    term = model.get_term_topics(l, minimum_probability=1e-8)#设置minimum_probability阈值，选出大于阈值的主题，输出[(主题编号，权重)...]
    i_max = term[0][1]
    label = 0
    for i in range(1, len(term)):
        #print(term[i][1])
        if (term[i][1] > i_max):
            i_max = term[i][1]
            label = i
    #print(dictionary[l]+" "+str(label))
    d1[dictionary[l]]=label
print(d1)#[(词，主题编号),()...]

#遍历paper.txt，每行属于同一个主题的分配到一个文件topic-i.txt中
for list1 in list:
    print(list1)
    for j in range(len(list1)):
        #print(list1[j])
        te=list1[j]
       # print(d1[list1[j]])
        l=d1[list1[j]]
        f=open("topic-"+str(l)+".txt","a")
        f.write(te+" ")
        f.close()
    for k in range(0,5):
         f=open("topic-"+str(k)+".txt","a")
         f.write("\n")
         f.close()


TypeError: decoding to str: need a bytes-like object, list found

In [ ]:
f = open("topic-4.txt")#以最后一个主题文件为例

line = f.readline()
list = []
while line:
    a = line.split()
    b = a[0:]
    if b==[]:
        line = f.readline()
        continue
    list.append(b)
    line = f.readline()
#print(list):[[],[],[]...]
f.close()

#转换为DataFrame格式
import pandas as pd

t_df = pd.DataFrame(list)
#print(t_df):18345*9,18345行中最长的一行有9个单词

#转换DataFrame数据为包含数据的列表
# df_arr = shopping_df.stack().groupby(level=0).apply(list).tolist()   # 方法一
def deal(data):
    return data.dropna().tolist()
df_arr = t_df.apply(deal, axis=1).tolist()  # 方法二
#print(df_arr)：[[],[],[]...]


from mlxtend.preprocessing import TransactionEncoder  # 传入模型的数据需要满足特定的格式，可以用这种方法来转换为bool值

te = TransactionEncoder()  # 定义模型
df_tf = te.fit_transform(df_arr)
df = pd.DataFrame(df_tf, columns=te.columns_)
#print(df):18345*2364，一共18345行，2364个单词

from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(df, min_support=0.002, use_colnames=True)  # use_colnames=True表示使用元素名字，默认的False使用列名代表元素
frequent_itemsets.sort_values(by='support', ascending=False, inplace=True)  # 频繁项集可以按支持度排序
print(frequent_itemsets)  # 打印频繁项集
frequent_itemsets.to_csv('G://pattern-4.txt', sep='\t', index=False, header=None)
